# Outages
**Name(s)**: Bill Wang, Ethan Cao

**Website Link**: (your website link)

## Code

In [1]:
import pandas as pd
import numpy as np
import os
import folium


import plotly.express as px
pd.options.plotting.backend = 'plotly'

In [2]:
def combine_times(date_col_name, time_col_name, new_col_name, df):
    df = df.copy()
    df[new_col_name] = df[date_col_name] + pd.to_timedelta(df[time_col_name].astype(str))
    return df

In [258]:
data = pd.read_excel("outage.xlsx", skiprows=[0,1,2,3,4,6], index_col=1).iloc[:,1:]
data = combine_times("OUTAGE.START.DATE", 'OUTAGE.START.TIME', 'OUTAGE.START.DATETIME', data)
data = combine_times("OUTAGE.RESTORATION.DATE", "OUTAGE.RESTORATION.TIME", "OUTAGE.RESTORATION.DATETIME", data)

In [259]:
data.columns

Index(['YEAR', 'MONTH', 'U.S._STATE', 'POSTAL.CODE', 'NERC.REGION',
       'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY',
       'OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE',
       'OUTAGE.RESTORATION.TIME', 'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL',
       'HURRICANE.NAMES', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW',
       'CUSTOMERS.AFFECTED', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE',
       'TOTAL.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES', 'TOTAL.SALES',
       'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN', 'RES.CUSTOMERS',
       'COM.CUSTOMERS', 'IND.CUSTOMERS', 'TOTAL.CUSTOMERS', 'RES.CUST.PCT',
       'COM.CUST.PCT', 'IND.CUST.PCT', 'PC.REALGSP.STATE', 'PC.REALGSP.USA',
       'PC.REALGSP.REL', 'PC.REALGSP.CHANGE', 'UTIL.REALGSP', 'TOTAL.REALGSP',
       'UTIL.CONTRI', 'PI.UTIL.OFUSA', 'POPULATION', 'POPPCT_URBAN',
       'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL',
       'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND', 'PCT_WATER_TOT',
       'PCT

In [260]:
data = data[['U.S._STATE',"YEAR", 'OUTAGE.START.DATETIME', 'OUTAGE.RESTORATION.DATETIME','CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL', 'CUSTOMERS.AFFECTED', 'OUTAGE.DURATION','DEMAND.LOSS.MW']]

In [190]:
univariant_plot = px.histogram(data['OUTAGE.DURATION'])
univariant_plot.update_layout(xaxis_title = 'OUTAGE.DURATION in Minutes', showlegend = False, title = 'Count of Duration of Outage')

In [191]:
univariant_plot.write_html('../uni-plot.html', include_plotlyjs='cdn')

In [192]:
bivariant = data.plot(kind = 'bar', x = 'U.S._STATE', y = 'OUTAGE.DURATION')

In [193]:
bivariant = data.groupby('U.S._STATE')['OUTAGE.DURATION'].sum().sort_values().reset_index().plot(kind = 'bar', x ='U.S._STATE' , y= 'OUTAGE.DURATION')

In [194]:
bivariant.update_xaxes(dtick=1)

In [195]:
bivariant.write_html('../bi-plot.html', include_plotlyjs='cdn')

In [196]:
data.groupby('U.S._STATE')['YEAR'].count().sort_values()

U.S._STATE
Alaska                    1
South Dakota              2
North Dakota              2
Montana                   3
Mississippi               4
West Virginia             4
Nebraska                  4
Hawaii                    5
Alabama                   6
Wyoming                   6
Nevada                    7
South Carolina            8
New Mexico                8
Iowa                      8
Vermont                   9
Idaho                     9
Kansas                    9
District of Columbia     10
Kentucky                 13
New Hampshire            14
Minnesota                15
Colorado                 15
Missouri                 17
Georgia                  17
Massachusetts            18
Connecticut              18
Maine                    19
Wisconsin                20
Oklahoma                 24
Arkansas                 25
Oregon                   26
Arizona                  28
Tennessee                34
New Jersey               35
Virginia                 37
Louisiana

In [197]:
data[['CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL']][data['CAUSE.CATEGORY.DETAIL'].isna()]

,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL
OBS,,
1,severe weather,NaN
5,severe weather,NaN
19,severe weather,NaN
20,severe weather,NaN
24,intentional attack,NaN
...,...,...
1523,system operability disruption,NaN
1525,public appeal,NaN
1530,public appeal,NaN


### Question
1. Out of Nerc Regions/ **States** , what are the most likely to have the worst outages, what are the causes.
2. UniVariant: 
3. BiVariant: 

In [198]:
# data[data[''].isnull()]

### Deadlines
1. Sat: Question, Cleaning & EDA, Plan for Univariant and Bivariant
2. Sun: Assignment of Missigness: Have Univariant and Bivariant done
3. Mon: Hypothesis Testing
4. Tue: Start the report 

In [199]:
data['CAUSE.CATEGORY'][data['CAUSE.CATEGORY'].isna()]

Series([], Name: CAUSE.CATEGORY, dtype: object)

### Cleaning and EDA

### Assessment of Missingness

## NMAR

**CAUSE CATEGORY**:
All the NAN values in cause category could nan because they could pinpoint the reason for the outage, therefore we can say that the missingness in the Cause Category must be dependent on itself. 

States and CUstomers.affected

## MAR

### US STATE WITH CUSTOMERS
**NULL HYPOTHESIS**: There is no significant difference between 

In [200]:
def find_tvd(df):
    df = df.copy()
    df_needed = df.loc[:,['U.S._STATE', 'CUSTOMERS.AFFECTED']]
    df_needed['ISNA'] = df_needed.loc[:,'CUSTOMERS.AFFECTED'].isna()
    df_needed['CUSTOMERS.AFFECTED'] = df_needed.loc[:,'CUSTOMERS.AFFECTED'].fillna(0)
    find_prop = df_needed.pivot_table(index = 'U.S._STATE', columns = 'ISNA', aggfunc = 'count', fill_value=0).loc[:,'CUSTOMERS.AFFECTED']
    total_not_missing = find_prop[False].sum()
    total_missing = find_prop[True].sum()
    find_prop[False] = find_prop[False] / total_not_missing
    find_prop[True] = find_prop[True] / total_missing
    return find_prop.diff(axis=1)[True].abs().sum()

In [201]:
def mar_permutation(df, N = 1000):
    observed = find_tvd(df)
    arr = []
    for _ in range(N):
        shuffled = df.assign(**{"CUSTOMERS.AFFECTED": np.random.permutation(df['CUSTOMERS.AFFECTED'])}) 
        arr.append(find_tvd(shuffled))
    plot = px.histogram(np.array(arr))
    plot.add_vline(x=observed, line_color= "green", annotation_text="obs")
    plot.update_layout(xaxis_title = "TVD of Missing and Non-Missing Values", yaxis_title = "Frequency", title_text = "Simulated Null Values",showlegend=False)


    return plot, np.array([np.array(arr) > observed]).mean()


In [202]:
graph, p_value = mar_permutation(data)
graph.write_html('../03-topic/mar-hist.html', include_plotlyjs='cdn')

### Hypothesis Testing

1. Out of Nerc Regions/ **States** , what are the most likely to have the worst outages, what are the causes.

In [203]:
data[['U.S._STATE', 'CUSTOMERS.AFFECTED']].groupby('U.S._STATE').mean()

,CUSTOMERS.AFFECTED
U.S._STATE,
Alabama,94328.800000
Alaska,14273.000000
Arizona,64402.666667
Arkansas,47673.846154
California,201365.716535
Colorado,41060.636364
Connecticut,60339.230769
Delaware,3475.000000
District of Columbia,194709.222222


In [205]:
def calculate_outage_severity(data, state, var):

    output = data.groupby(state).mean()
    
    return output.loc[True,var]

In [206]:
def perm_test(data, state, var, n=1000):
    data = data.copy()[["U.S._STATE", var]]
    data[state] = data["U.S._STATE"] == state
    obs = calculate_outage_severity(data, state, var)
    test_stats = []
    for _ in range(n):
        value = np.random.permutation(data[var])
        shuffled = data.assign(**{var : value })
        trial = calculate_outage_severity(shuffled, state, var)
        test_stats.append(trial)
    
    return  (np.array(test_stats) >= obs).mean()

**This** represents texas has a difference. 

In [207]:
stat = perm_test(data, "Texas", 'OUTAGE.DURATION')
stat

0.419

In [208]:
all_p_values_duration = {}
all_p_values_customers = {}
for i in data['U.S._STATE'].unique():
    all_p_values_duration[i] = perm_test(data, i, "OUTAGE.DURATION")
    all_p_values_customers[i] = perm_test(data, i , "CUSTOMERS.AFFECTED")

create dataframe to look p_value and how many outages to find how likely an outage is gonna happen and how severe it is. 

In [ ]:
initial = pd.DataFrame({'duration_p_value' : pd.Series(all_p_values_duration),
              'customer_p_value' : pd.Series(all_p_values_customers)})
resulting_df = data.groupby('U.S._STATE')[['YEAR']].count().merge(initial, left_index = True, right_index = True).rename(columns = {'YEAR': 'Count of Outages'})
graph_df = resulting_df[(resulting_df['duration_p_value'] < 0.05) | (resulting_df['customer_p_value'] < 0.05 )].sort_values('customer_p_value')

In [382]:
graph_df

,Count of Outages,duration_p_value,customer_p_value
Montana,3,0.979,0.000
South Dakota,2,0.893,0.000
Texas,127,0.372,0.010
California,210,1.000,0.015
Florida,45,0.069,0.017
New York,71,0.000,0.115
West Virginia,4,0.049,0.212
Michigan,95,0.001,0.367
Alaska,1,0.000,0.498
Wisconsin,20,0.005,0.994


In [384]:
fig = px.bar(graph_df.reset_index().rename(columns = {'index': 'U.S._STATES'}), x='U.S._STATES', y=['duration_p_value', 'customer_p_value'], title='Grouped Bar Chart of Duration P-Value and Customer P-Value by U.S. States with a P-Value less than 0.05')
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig

In [385]:
fig.write_html('../p_value_bar.html', include_plotlyjs='cdn')

In [306]:
compare_df = pd.DataFrame(all_p_values_duration, index = np.arange(0,50)).melt().groupby('variable').max().sort_values('value')

In [214]:
compare_df#.merge(pd.DataFrame(all_p_values_customers, index = np.arange(0,50)).melt().groupby('variable').max().sort_values('value'))

,value
variable,
Alaska,0.000
New York,0.000
Michigan,0.001
Wisconsin,0.005
West Virginia,0.049
Arizona,0.056
New Jersey,0.067
Florida,0.069
Louisiana,0.080


In [215]:
compare_df = compare_df.merge(data.groupby('U.S._STATE')['YEAR'].count().sort_values(ascending= False),left_index = True, right_index = True)

In [226]:
customer_df =pd.DataFrame(all_p_values_customers, index = np.arange(0,50)).T[[0]].rename(columns = {0: 'customer_p_value'})

In [227]:
customer_df

,customer_p_value
Minnesota,0.435
Tennessee,0.991
Wisconsin,0.994
West Virginia,0.212
Michigan,0.367
Texas,0.010
Indiana,0.971
Alabama,0.565
Mississippi,0.958
Illinois,0.107


In [229]:
customer_duration_df = compare_df.merge(customer_df, left_index= True, right_index= True)

In [232]:
avg_dur = data['OUTAGE.DURATION'].mean()
avg_cus = data['CUSTOMERS.AFFECTED'].mean()


dir_duration = data.groupby("U.S._STATE")['OUTAGE.DURATION'].agg(lambda ser: ser.mean() - avg_dur)
dir_customer = data.groupby("U.S._STATE")['CUSTOMERS.AFFECTED'].agg(lambda ser: ser.mean() - avg_cus)

In [309]:
amount_outages = customer_duration_df.rename(columns = {"YEAR": 'amount_outage'}).sort_values(by="value", ascending=True)
item = amount_outages#[amount_outages["amount_outage"] > 10]

item["dir_duration"] = dir_duration
item['dir_customer'] = dir_customer
item = item.reset_index().rename(columns = {"index" : 'U.S._STATE'}).fillna(0).sort_values(by = 'customer_p_value')

In [315]:
resulting_df = item.set_index('U.S._STATE')[['value','customer_p_value']].rename(columns= {'value':'duration_p_value'}).sort_index()

In [338]:
graph_df = resulting_df[(resulting_df['duration_p_value'] < 0.05) | (resulting_df['customer_p_value'] < 0.05 )].sort_values('customer_p_value')

In [339]:
graph_df

,duration_p_value,customer_p_value
U.S._STATE,,
Montana,0.979,0.000
South Dakota,0.893,0.000
Texas,0.372,0.010
California,1.000,0.015
Florida,0.069,0.017
New York,0.000,0.115
West Virginia,0.049,0.212
Michigan,0.001,0.367
Alaska,0.000,0.498


In [383]:
fig = px.bar(graph_df.reset_index(), x='U.S._STATE', y=['duration_p_value', 'customer_p_value'], title='Grouped Bar Chart of Duration P-Value and Customer P-Value by U.S. States with a P-Value less than 0.05')
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['index', 'Count of Outages', 'duration_p_value', 'customer_p_value'] but received: U.S._STATE

In [290]:
# fig = px.bar(item, x='U.S._STATE', y=['value', 'customer_p_value'], title='Grouped Bar Chart of Value and Amount_Outage by U.S. States')
# fig.update_layout(barmode='group', xaxis_tickangle=-45)
# fig.show()

In [291]:
demand_loss_mean = data.groupby('U.S._STATE')['DEMAND.LOSS.MW'].mean().fillna(0)
item = item.merge(demand_loss_mean, left_on= 'U.S._STATE', right_index = True)

In [294]:
fig = px.bar(item, x='U.S._STATE', y=['value', 'customer_p_value'], title='Grouped Bar Chart of Value and Amount_Outage by U.S. States')
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [284]:
item

,U.S._STATE,value,amount_outage,customer_p_value,dir_duration,dir_customer,DEMAND.LOSS.MW
28,South Dakota,0.893,2,0.000,-2505.398374,0.000000,228.500000
36,Montana,0.979,3,0.000,-2571.398374,0.000000,0.000000
20,Texas,0.372,127,0.010,79.421298,79775.873013,552.083333
49,California,1.000,210,0.015,-959.059990,57909.493804,667.594937
7,Florida,0.069,45,0.017,1469.268293,145912.868178,804.575000
30,Illinois,0.934,46,0.107,-1022.943829,63570.936359,214.222222
1,New York,0.000,71,0.115,3409.558769,47219.643935,1283.147059
17,South Carolina,0.266,8,0.118,509.601626,108456.902269,1699.714286
12,District of Columbia,0.125,10,0.180,1678.201626,51252.999491,1280.000000
4,West Virginia,0.049,4,0.212,4353.601626,36338.110602,362.000000


In [285]:
# # Example weights for components
# weight_outage_duration = 0.4
# weight_amount_duration = 0.3
# weight_customer_pvalue = 0.3




# # Calculate the weighted mean composite score
# item['Composite_Score'] =  (item['DEMAND.LOSS.MW'] * amount_outages)
# item

In [286]:
item#.sort_values(by = 'Composite_Score').plot(kind = 'bar', x= 'U.S._STATE', y = "Composite_Score")

,U.S._STATE,value,amount_outage,customer_p_value,dir_duration,dir_customer,DEMAND.LOSS.MW
28,South Dakota,0.893,2,0.000,-2505.398374,0.000000,228.500000
36,Montana,0.979,3,0.000,-2571.398374,0.000000,0.000000
20,Texas,0.372,127,0.010,79.421298,79775.873013,552.083333
49,California,1.000,210,0.015,-959.059990,57909.493804,667.594937
7,Florida,0.069,45,0.017,1469.268293,145912.868178,804.575000
30,Illinois,0.934,46,0.107,-1022.943829,63570.936359,214.222222
1,New York,0.000,71,0.115,3409.558769,47219.643935,1283.147059
17,South Carolina,0.266,8,0.118,509.601626,108456.902269,1699.714286
12,District of Columbia,0.125,10,0.180,1678.201626,51252.999491,1280.000000
4,West Virginia,0.049,4,0.212,4353.601626,36338.110602,362.000000


In [300]:
df_both = item[(item['value'] < 0.05) | (item['customer_p_value'] < 0.05 )]
df_both

,U.S._STATE,value,amount_outage,customer_p_value,dir_duration,dir_customer,DEMAND.LOSS.MW
28,South Dakota,0.893,2,0.000,-2505.398374,0.000000,228.500000
36,Montana,0.979,3,0.000,-2571.398374,0.000000,0.000000
20,Texas,0.372,127,0.010,79.421298,79775.873013,552.083333
49,California,1.000,210,0.015,-959.059990,57909.493804,667.594937
7,Florida,0.069,45,0.017,1469.268293,145912.868178,804.575000
1,New York,0.000,71,0.115,3409.558769,47219.643935,1283.147059
4,West Virginia,0.049,4,0.212,4353.601626,36338.110602,362.000000
2,Michigan,0.001,95,0.367,2677.580573,9422.021713,696.549020
0,Alaska,0.000,1,0.498,0.000000,-129183.222731,35.000000
3,Wisconsin,0.005,20,0.994,5278.706889,-97580.222731,161.000000


In [303]:
fig = px.bar(df_both, x='U.S._STATE', y=['value', 'customer_p_value'], title='Grouped Bar Chart of Value and Amount_Outage by U.S. States')
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig

In [305]:
fig.write_html('../p_value_bar.html', include_plotlyjs='cdn')